<span style="float:left;">Licence CC BY-NC-ND</span><span style="float:right;">François Rechenmann &amp; Thierry Parmentelat&nbsp;<img src="media/inria-25.png" style="display:inline"></span><br/>

# Algorithme UPGMA

Nous allons voir dans ce complément une implémentation en python de l'algorithme UPGMA qui a vient d'être expliqué dans la vidéo. Mais comme toujours commençons par ceci. 

In [ ]:
# la formule magique pour utiliser print() en python2 et python3
from __future__ import print_function
# pour que la division se comporte en python2 comme en python3
from __future__ import division

### Distance de Needleman et Wunsch

Nous avons bien sûr besoin de la fonction `distance` de Needleman et Wunsch&nbsp;:

In [ ]:
from w4_s09_c1_needleman_wunsh_iter import needleman_wunsch, distance

### Format du fichier d'entrée

Nous allons cette fois utiliser un fichier d'entrée dans un format légèrement différent, de façon à pouvoir associer de façon simple un nom aux différents fragments d'ADN que nous manipulons. Pour cela, le fichier d'entrée va ressembler à ceci (ce sont bien sûr des données fantaisistes pour commencer)&nbsp;: 

In [ ]:
cat data/named-species.txt

C'est à dire que chaque ligne contient maintenant, séparé par un ou plusieurs espaces, un nom et une séquence d'ADN. Pour cette raison nous ne pouvons pas réutiliser telle quelle la fonction qui élabore le tableau des distances, nous allons récrire cette partie.

### La méthode `split`

Nous allons utiliser la métode `split` sur les chaines en python, qui découpe simplement une chaine en morceaux&nbsp&nbsp;:

In [ ]:
chaine = "BABAAB  ADAD"
chaine.split()

Comme on le voit, sans argument cette méthode fait exactement ce dont nous allons avoir besoin pour découper une ligne d'entrée en deux partie. Pour information, on peut passer à `split` le séparateur à utiliser pour le découpage&nbsp;:

In [ ]:
chaine.split("A")

### La méthode `remove` pour enlever d'une liste

Nous utilisons également la méthode `remove` sur les listes, qui fonctionne un peu comme `append`, mais a l'effet inverse d'enlever un élément d'une liste&nbsp;:

In [ ]:
# une liste avec des chaines et des tuples
l = [ 'a', 'b', (1, 2)]
print("début:", l)
# avec remove on peut enlever une chaine
l.remove('a')
print("milieu:", l)
# ou on peut enlever un tuple
l.remove( (1, 2))
print("fin:", l)


### Structures de données

Pour écrire l'algorithme nous allons utiliser les structures de données suivantes. 

##### Espèces

Pour représenter les différentes espèces qui entrent en jeu, nous allons utiliser&nbsp;:
  * soit directement une chaine, pour les espèces qui sont présentes dans le fichier d'entrée, dites espèces **natives**,
  * soit un tuple de deux espèces pour les espèces qui sont créés par l'algorithme au fur et à mesure de sa progression, qu'on appellera des **espèces synthétiques**.

Ainsi par exemple une espèce pourra être 
 * soit `spam`,
 * soit `('eggs', 'bread')` pour représenter l'ancêtre commun aux espèces `eggs` et `bread`,
 * soit encore un étage au-dessus `(('eggs', 'bread'), ('bacon', 'chicken'))`

##### `native_species`

Cette variable est simplement le résultat de l'analyse du fichier d'entrée, c'est simplement un dictionnaire `name` $\rightarrow$ `adn`.

##### `distances`

Exactement comme on l'a vu dans le complément précédent sur le tableau des distances, cette variable va mémoriser les distances entre deux espèces, natives ou synthétiques. Comme dans ce cas-là, ceci sera implémenté comme un dictionnaire indexé sur un tuple d'espèces, et on évite la duplication au moyen d'une fonction de commodité `get_distance`. Évidemment on l'initialise avec uniquement les espèces natives, puis on ajoute au fur et à mesure toutes les distances nécessaires.

##### `species`

Cette variable jour un rôle clé, c'est la liste des espèces qui sont encore *en lice*. On l'initialise avec les espèces natives, et à chaque création d'une espèce synthétique on **ajoute la nouvelle espèce** et on **enlève les deux espèces filles** (natives ou synthétiques) qui la composent.

C'est donc cette variable qui détermine la fin de l'algorithme (lorsqu'il n'y a plus qu'une espèce), et qui à ce stade contient donc le résultat final.

***

XXX rédaction à terminer

***

### Commodité pour accéder aux distances

In [ ]:
def get_distance(distances, k1, k2):
    if k1 == k2:
        return 0
    elif (k1, k2) in distances:
        return distances[ (k1, k2) ]
    else:
        return distances[ (k2, k1)]

### Calcul de la distance minimale

In [ ]:
def minimal_couple(distances, species):
    couple, min_value = None, 10**100
    for k1 in species:
        for k2 in species:
            if k1 == k2:
                continue
            if get_distance(distances, k1, k2) < min_value:
                min_value = get_distance(distances, k1, k2)
                couple = k1, k2
    return couple

### L'algorithme UPGMA

In [ ]:
def UPGMA(filename, verbose=False):
    """
    Lit un fichier contenant sur chaque ligne 
    un nom d'espèce et un ADN

    Calcule le tableau des distances, 
    puis implémente l'algorithme UPGMA
    
    Renvoie l'arbre de filiation sous forme d'un tuple 
    sur les noms d'espèces
    """
    
    native_species = {}

    # lire le fichier

    with open(filename) as input:
        for line in input:
            name, adn = line.split()
            native_species[name] = adn
    
    # on calcule le tableau des distances
    distances = {}
    
    for name1, adn1 in native_species.items():
        for name2, adn2 in native_species.items():
            if name1 == name2:
                continue
            key = (name1, name2)
            rkey = (name2, name1)
            if rkey in distances:
                continue
            distances[key] = distance(adn1, adn2)
    
    # la liste des clés de départ
    species = native_species.keys()
    
    if verbose:
        print(10*'+', 'Initial distances')
        print(distances )
    while len(species) > 1:
        bro1, bro2 = minimal_couple(distances, species)
        new_key = bro1, bro2
        species.remove(bro1)
        species.remove(bro2)
        for old_key in species:
            # dist(F,C),A = (dist F,A + dist C,A) / 2 
            distances[ (old_key, new_key) ] = \
              (get_distance(distances, bro1, old_key) + \
               get_distance(distances, bro2, old_key)) / 2
        species.append(new_key)
        if verbose:
            print(10*'=', "species = ", species)
            print(distances)
    return species[0]

In [ ]:
UPGMA("data/named-species.txt", True)